In [ ]:
## Note: seismic_profile, RMS_amplitude, first derivative, generalized spectral decomposition, 
import numpy as np
import pandas as pd 
import seaborn as sns 
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.decomposition import PCA
from PIL import Image
import segyio
# import cv2 

import os

In [ ]:
path = '/Users/rishikhare/Downloads/DOI-SeismicFacies/Seismic_files'

In [ ]:
data_types = []
for file in os.listdir(path):
  data_types.append(file)

feature_list = ['local_structural_dip', 'RMS_amplitude', 'generalized_spectral_decomposition', 'variance', 'flatness', 'dominant_freq', 'seismic_profile', 'first_derivative', 
                'instantaneous_phase', 'instantaneous_freq', 'amplitude_contrast', 'chaos', 'iso_freq_54hz', 'gradient_magnitude']

data_types

In [ ]:
temp = []

for file in os.listdir(path):
  with segyio.open(os.path.join(path, file), strict=False) as s:
    _data = np.stack(t.astype(float) for t in s.trace)
  temp.append(_data)

tuples = [(key, value) for i, (key, value) in enumerate(zip(feature_list, temp))]
data = dict(tuples)

In [ ]:
seis_df_v = pd.DataFrame(data['seismic_profile'].T)
plt.subplot(1,2,1)
vertical_slice = seis_df_v[0].plot.line(figsize=(15, 5))
plt.title('Vertical Slice (index 0)')

seis_df_h = pd.DataFrame(data['seismic_profile'])
plt.subplot(1,2,2)
horizontal_slice = seis_df_h[600].plot.line(figsize=(15,5))
plt.title('Horizontal Slice (index 600)')



In [ ]:
n = 10
fig, axs = plt.subplots(n, figsize=(20,20))
fig.suptitle('Vertical Slices')
for i in range(n):
    axs[i].plot(seis_df_v[i].values)
    axs[i].set_title('index ' + str(i))
fig.tight_layout(pad=2.0)
plt.show()

In [ ]:
n = 10
fig, axs = plt.subplots(n, figsize=(25,25))
fig.suptitle('Horizontal Slices')
for i in range(n):
    axs[i].plot(seis_df_h[i+600].values)
    axs[i].set_title('index ' + str(i+600))
fig.tight_layout(pad=2.0)
plt.show()

In [ ]:
_dfs = []

for key in data.keys():
  _df = pd.DataFrame(data[key]).stack().rename_axis(['X', 'Y']).reset_index(name=key)
  _dfs.append(_df)

In [ ]:
dfs = [df.set_index(['X', 'Y']) for df in _dfs]
final_df = pd.concat(dfs, axis=1).reset_index()

In [ ]:
pd.set_option('float_format', '{:f}'.format)
final_df.describe()

In [ ]:
pca = PCA()
principle_components = pca.fit_transform(final_df[feature_list])
pca_df = pd.DataFrame(data=principle_components)

pca_df

In [ ]:
variance_ratios = pca.explained_variance_ratio_
plt.bar(range(len(variance_ratios)), variance_ratios)
plt.title('Percentage Variance Explained by Component')
plt.xlabel('Components')
plt.xticks(np.arange(0,14))
plt.ylabel('Percentage Variance')

In [ ]:
plt.scatter(pca_df.iloc[:,0], pca_df.iloc[:,1], s=1)

In [ ]:
scaler = preprocessing.StandardScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(final_df[feature_list]), columns=feature_list)
features_scaled

In [ ]:
scaled_pca = PCA()
scaled_principle_components = scaled_pca.fit_transform(features_scaled)
scaled_pca_df = pd.DataFrame(data=scaled_principle_components)
scaled_pca_df

In [ ]:
scaled_variance_ratios = pca.explained_variance_ratio_
plt.bar(range(len(scaled_variance_ratios)), scaled_variance_ratios)
plt.title('Percentage Variance Explained by Component')
plt.xlabel('Components')
plt.xticks(np.arange(0,14))
plt.ylabel('Percentage Variance')

In [ ]:
plt.scatter(scaled_pca_df.iloc[:,0], scaled_pca_df.iloc[:,1], s=1)

In [ ]:
# Set data clip for better viewing
clip_percentile = 96
vm = np.percentile(data['seismic_profile'].T, clip_percentile)

# Plot with Matplotlib
plt.figure(figsize=(25,10))
plt.axes().set_axis_off()                                      
plt.imshow(data['seismic_profile'].T, cmap="gray", vmin=-vm, vmax=vm, aspect='auto')           
plt.ylim([1000, 0])                                             
plt.savefig('WNC82_017_seismic_profile', bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
from skimage.feature import canny
from skimage.filters import sobel
from skimage import data,morphology
from skimage.segmentation import watershed
from skimage.color import rgb2gray,label2rgb
import scipy.ndimage as nd

# load images and convert grayscale
img = np.asarray(Image.open('WNC82_017_seismic_profile.png'))
im = cv2.imread('WNC82_017_seismic_profile.png')
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
WNC82_017 = im

# apply edge segmentation
# plot canny edge detection
edges = canny(WNC82_017)
plt.imshow(edges, interpolation='gaussian')
plt.title('Canny detector')

# fill regions to perform edge segmentation
fill_im = nd.binary_fill_holes(edges)
plt.imshow(fill_im)
plt.title('Region Filling')

# Region Segmentation
# First we print the elevation map
elevation_map = sobel(WNC82_017)
plt.imshow(elevation_map)

# Since, the contrast difference is not much. Anyways we will perform it
markers = np.zeros_like(WNC82_017)
markers[WNC82_017 < 0.1171875] = 1 # 30/255
markers[WNC82_017 > 0.5859375] = 2 # 150/255

plt.imshow(markers)
plt.title('markers')

# Perform watershed region segmentation
segmentation = watershed(elevation_map, markers)

plt.imshow(segmentation)
plt.title('Watershed segmentation')

# plot overlays and contour
segmentation = nd.binary_fill_holes(segmentation - 1)
label_rock, _ = nd.label(segmentation)
# overlay image with different labels
image_label_overlay = label2rgb(label_rock, image=WNC82_017)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 16), sharey=True)
ax1.imshow(WNC82_017)
ax1.contour(segmentation, [0.8], linewidths=1.8, colors='w')
ax2.imshow(image_label_overlay)



In [ ]:
# Importing required boundaries
from skimage.segmentation import slic, mark_boundaries

# Setting the plot figure as 15, 15
plt.figure(figsize=(15, 15))

# Sample Image of scikit-image package
img = WNC82_017 

# Applying SLIC segmentation
# for the edges to be drawn over
segments = slic(img, n_segments=100, compactness=1)

plt.subplot(1, 2, 1)

# Plotting the original image
plt.imshow(img)

# Detecting boundaries for labels
plt.subplot(1, 2, 2)

# Plotting the output of marked_boundaries
# function i.e. the image with segmented boundaries
plt.imshow(mark_boundaries(img, segments))



In [ ]:
# Importing required libraries
from skimage.segmentation import slic
from skimage.color import label2rgb

# Setting the plot size as 15, 15
plt.figure(figsize=(15,15))

# Sample Image of scikit-image package
img = WNC82_017

# Applying Simple Linear Iterative
# Clustering on the image
# - 50 segments & compactness = 10
img_segments = slic(img,
						n_segments=50,
						compactness=10)
plt.subplot(1,2,1)

# Plotting the original image
plt.imshow(img)
plt.subplot(1,2,2)

# Converts a label image into
# an RGB color image for visualizing
# the labeled regions.
plt.imshow(label2rgb(img_segments,
					img,
					kind = 'avg'))



In [ ]:
# SEGMENTATION
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('WNC82_017_seismic_profile.png')
b,g,r = cv2.split(img)
rgb_img = cv2.merge([r,g,b])
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# noise removal
kernel = np.ones((2,2),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(closing,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(sure_bg,cv2.DIST_L2,3)
# Threshold
ret, sure_fg = cv2.threshold(dist_transform,0.1*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0
markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]
plt.subplot(211),plt.imshow(rgb_img)
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(212),plt.imshow(thresh, 'gray')
# plt.imsave(r'thresh.png',thresh)
plt.title("Otsu's binary threshold"), plt.xticks([]), plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
from skimage.filters import gaussian
gaussian_blur = gaussian(WNC82_017, sigma=1)

In [ ]:
plt.imshow(gaussian_blur)

In [ ]:
from sklearn.cluster import KMeans

for k in np.arange(10):
  X = ??
  kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(X)
  kmeans.labels_

In [ ]:
kmeans.predict([[0, 0], [12, 3]])

In [ ]:
kmeans.cluster_centers_